<a href="https://colab.research.google.com/github/19xx47/19xx47/blob/main/Lab6_supervised_classification_LU_Phuket.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://githubtocolab.com/giswqs/geemap/blob/master/examples/notebooks/32_supervised_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

Uncomment the following line to install [geemap](https://geemap.org) if needed.

In [ ]:
!pip install geemap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 KB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.6/99.6 KB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 KB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 KB 18.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 KB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

# Machine Learning with Earth Engine - Supervised Classification

## Supervised classification algorithms available in Earth Engine

Source: https://developers.google.com/earth-engine/classification

The `Classifier` package handles supervised classification by traditional ML algorithms running in Earth Engine. These classifiers include CART, RandomForest, NaiveBayes and SVM. The general workflow for classification is:

1. Collect training data. Assemble features which have a property that stores the known class label and properties storing numeric values for the predictors.
2. Instantiate a classifier. Set its parameters if necessary.
3. Train the classifier using the training data.
4. Classify an image or feature collection.
5. Estimate classification error with independent validation data.

The training data is a `FeatureCollection` with a property storing the class label and properties storing predictor variables. Class labels should be consecutive, integers starting from 0. If necessary, use remap() to convert class values to consecutive integers. The predictors should be numeric.

![](https://i.imgur.com/vROsEiq.png)

## Step-by-step tutorial

### Import libraries

In [ ]:
import ee
import geemap

### Create an interactive map

In [ ]:
Map = geemap.Map()
Map

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=78flM-b_a00HCtwgVcF34LesUAbAhPXmM7RrGM8j_IY&tc=tdUZFRgFeDgJw7jL0G6lDbpLaiwCNS1t8aJXET024Ms&cc=1UIBgOFIrAvCpD4cbggOHKx6eRMtSfxBEnw7anvHUsE

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AWtgzh4OrAI6E4KLFfGU4UnIg4aqttl2ZMnZmwWwMvWAooZ4_TXI9lGZOUg

Successfully saved authorization token.


Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

### Add data to the map

In [ ]:
point = ee.Geometry.Point([98.43, 8])
# point = ee.Geometry.Point([-87.7719, 41.8799])

image = (
    ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
    .filterBounds(point)
    .filterDate('2016-01-01', '2016-12-31')
    .sort('CLOUD_COVER')
    .first()
    .select('B[1-7]')
)

vis_params = {'min': 0, 'max': 3000, 'bands': ['B5', 'B4', 'B3']}

Map.centerObject(point, 8)
Map.addLayer(image, vis_params, "Landsat-8")

### Check image properties

In [ ]:
ee.Date(image.get('system:time_start')).format('YYYY-MM-dd').getInfo()

'2016-01-14'

In [ ]:
image.get('CLOUD_COVER').getInfo()

1.88

### Make training dataset

There are several ways you can create a region for generating the training dataset.

- Draw a shape (e.g., rectangle) on the map and the use `region = Map.user_roi`
- Define a geometry, such as `region = ee.Geometry.Rectangle([-122.6003, 37.4831, -121.8036, 37.8288])`
- Create a buffer zone around a point, such as `region = ee.Geometry.Point([-122.4439, 37.7538]).buffer(10000)`
- If you don't define a region, it will use the image footprint by default

In [ ]:
region = Map.user_roi
# region = ee.Geometry.Rectangle([-122.6003, 37.4831, -121.8036, 37.8288])
# region = ee.Geometry.Point([98.43, 8]).buffer(1000)

In this example, we are going to use the [Copernicus Global Land Cover Layers: CGLS-LC100 Collection 3] https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_Landcover_100m_Proba-V-C3_Global#bands to create label dataset for training

In [ ]:
corine = ee.Image("COPERNICUS/Landcover/100m/Proba-V-C3/Global/2019").select('discrete_classification').clip(image.geometry())
Map.addLayer(corine, {}, 'CORINE')
Map

Map(bottom=125573.09094238281, center=[7.938072139300737, 98.67231263920132], controls=(WidgetControl(options=…

In [ ]:
# Make the training dataset.
points = corine.sample(
    **{
        'region': image.geometry(),
        'scale': 100,
        'numPixels': 5000,
        'seed': 0,
        'geometries': True,  # Set this to False to ignore geometries
    }
)

Map.addLayer(points, {}, 'training', False)

In [ ]:
print(points.size().getInfo())

5000


In [ ]:
print(points.first().getInfo())

{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [98.33669283371847, 8.763906574104801]}, 'id': '0', 'properties': {'discrete_classification': 112}}


### Train the classifier

In [ ]:
# Use these bands for prediction.
bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']


# This property of the table stores the land cover labels.
label = 'discrete_classification'

# Overlay the points on the imagery to get training.
training = image.select(bands).sampleRegions(
    **{'collection': points, 'properties': [label], 'scale': 30}
)

# Train a CART classifier with default parameters.
trained = ee.Classifier.smileCart().train(training, label, bands)

In [ ]:
print(training.first().getInfo())

{'type': 'Feature', 'geometry': None, 'id': '0_0', 'properties': {'B1': 105, 'B2': 152, 'B3': 405, 'B4': 251, 'B5': 3313, 'B6': 1671, 'B7': 646, 'discrete_classification': 112}}


### Classify the image

In [ ]:
# Classify the image with the same bands used for training.
result = image.select(bands).classify(trained)

# # Display the clusters with random colors.
Map.addLayer(result.randomVisualizer(), {}, 'classified')
Map

Map(bottom=31607.0, center=[7.999999999999998, 98.43000000000002], controls=(WidgetControl(options=['position'…

### Render categorical map

To render a categorical map, we can set two image properties: `landcover_class_values` and `landcover_class_palette`. We can use the same style as the NLCD so that it is easy to compare the two maps. 

In [ ]:
class_values = corine.get('discrete_classification_class_values').getInfo()
class_values

[0,
 20,
 30,
 40,
 50,
 60,
 70,
 80,
 90,
 100,
 111,
 112,
 113,
 114,
 115,
 116,
 121,
 122,
 123,
 124,
 125,
 126,
 200]

In [ ]:
class_palette = corine.get('discrete_classification_class_palette').getInfo()
class_palette

['282828',
 'FFBB22',
 'FFFF4C',
 'F096FF',
 'FA0000',
 'B4B4B4',
 'F0F0F0',
 '0032C8',
 '0096A0',
 'FAE6A0',
 '58481F',
 '009900',
 '70663E',
 '00CC00',
 '4E751F',
 '007800',
 '666000',
 '8DB400',
 '8D7400',
 'A0DC00',
 '929900',
 '648C00',
 '000080']

In [ ]:
landcover = result.set('classification_class_values', class_values)
landcover = landcover.set('classification_class_palette', class_palette)

In [ ]:
Map.addLayer(landcover, {}, 'Land cover')
Map

Map(bottom=31607.0, center=[7.999999999999998, 98.43000000000002], controls=(WidgetControl(options=['position'…

### Visualize the result

In [ ]:
print('Change layer opacity:')
cluster_layer = Map.layers[-1]
cluster_layer.interact(opacity=(0, 1, 0.1))

Change layer opacity:


Box(children=(FloatSlider(value=1.0, description='opacity', max=1.0),))

### Add a legend to the map

In [ ]:
Map.add_legend(builtin_legend='COPERNICUS/Landcover/100m/Proba-V/Global')
Map

Map(bottom=501390.5, center=[7.93870588196828, 98.47509437215425], controls=(WidgetControl(options=['position'…

### Export the result

Export the result directly to your computer:

In [ ]:
import os

out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
out_file = os.path.join(out_dir, 'landcover.tif')

In [ ]:
geemap.ee_export_image(landcover, filename=out_file, scale=900)

Generating URL ...
Please wait ...
An error occurred while downloading.


JSONDecodeError: ignored

Export the result to Google Drive:

In [ ]:
geemap.ee_export_image_to_drive(
    landcover, description='landcover', folder='export', scale=900
)